# Data Preparation

## Imports

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
import pandas as pd
import os

## Tratar Valores Ausentes

### FF_MED.csv

Quando a percentagem de valores ausentes for superior a 30% em cada estação, usamos interpolação linear; nas restantes utilizámos a mediana

In [ ]:
# Caminho para o dataset
dataset_path = os.path.join('data', 'raw', 'Dados_diarios_T_Vento_Prec_capitaisDistrito_2013 a 2023.xlsx')

# Importando a folha 'FF_MED', tratando -990 como ausente
df = pd.read_excel(dataset_path, sheet_name='FF_MED', na_values=-990)

# Resumo dos valores ausentes
missing_data = pd.DataFrame({
    'Total Ausentes': df.isnull().sum(),
    'Percentual (%)': (df.isnull().mean() * 100).round(2)
})

print("\nResumo das colunas com valores ausentes:")
print(missing_data)



Resumo das colunas com valores ausentes:
         Total Ausentes  Percentual (%)
ANO                   0            0.00
MES                   0            0.00
DIA                   0            0.00
1200551            1981           49.32
1210622            1525           37.96
1200567              14            0.35
1200575              57            1.42
1200545             283            7.05
1210702             663           16.50
1200560              83            2.07
1210683             190            4.73
1200548              57            1.42
1200570               1            0.02
1210718             191            4.75
1210734              52            1.29
1200571             226            5.63
1200579              32            0.80
1210770             192            4.78
1200558              28            0.70
1200562             471           11.73
1200554             352            8.76


In [43]:
missing_percent = (df.isnull().mean() * 100)

# Fazendo uma cópia do dataframe original para tratamento
df_treated = df.copy()

# Tratamento:
for col in df.columns:
    if missing_percent[col] > 30:
        # Se mais de 30% de ausentes → interpolar
        df_treated[col] = df_treated[col].interpolate(method='linear', limit_direction='both')
        # Preencher o que ainda sobrou (bordas) com a mediana
        mediana = df_treated[col].median()
        df_treated[col] = df_treated[col].fillna(mediana)
    elif missing_percent[col] < 30:
        # Se menos de 5% de ausentes → preencher com mediana
        mediana = df_treated[col].median()
        df_treated[col] = df_treated[col].fillna(mediana)
    else:
        pass

# Resultado final
print("\nResumo dos valores ausentes após tratamento:")
print(df_treated.isnull().sum())


Resumo dos valores ausentes após tratamento:
ANO        0
MES        0
DIA        0
1200551    0
1210622    0
1200567    0
1200575    0
1200545    0
1210702    0
1200560    0
1210683    0
1200548    0
1200570    0
1210718    0
1210734    0
1200571    0
1200579    0
1210770    0
1200558    0
1200562    0
1200554    0
dtype: int64


In [44]:
# Salvando o dataset tratado em CSV
df_treated.to_csv('data/processed/FF_MED_processed.csv', index=False)


### TT_MED.csv
Substitui os valores ausentes pela temperatura média

In [51]:
import pandas as pd
import os

# Caminho para o dataset
dataset_path = os.path.join('data', 'raw', 'Dados_diarios_T_Vento_Prec_capitaisDistrito_2013 a 2023.xlsx')

# Importando a folha 'TT_MED', tratando -990 como ausente
df_ttmed = pd.read_excel(dataset_path, sheet_name='T_MED', na_values=-990)

# Verificando se existem valores ausentes antes de qualquer tratamento
print("\nResumo dos valores ausentes antes do tratamento na folha TT_MED:")
print(df_ttmed.isnull().sum())

# Se houver valores ausentes, vamos prosseguir com o tratamento:
if df_ttmed.isnull().sum().sum() > 0:
    # Calculando a média global (média de todos os valores numéricos do dataframe)
    media_global = df_ttmed.select_dtypes(include=['float64', 'int64']).stack().mean()

    # Preenchendo valores ausentes com a média global
    df_ttmed.fillna(media_global, inplace=True)

    # Resultado final após o tratamento
    print("\nResumo dos valores ausentes após o tratamento na folha TT_MED:")
    print(df_ttmed.isnull().sum())



    # Salvando o dataset tratado em Excel (opcional)
    df_ttmed.to_csv('data/processed/T_MED_processed.csv', index=False)
else:
    print("\nNão há valores ausentes na folha TT_MED.")



Resumo dos valores ausentes antes do tratamento na folha TT_MED:
ANO          0
MES          0
DIA          0
1200551    288
1210622    240
1200567     14
1200575     49
1200545    512
1210702     98
1200560     32
1210683    198
1200548     94
1200570      5
1210718    343
1210734     48
1200571    332
1200579     51
1210770    297
1200558     30
1200562    594
1200554     31
dtype: int64

Resumo dos valores ausentes após o tratamento na folha TT_MED:
ANO        0
MES        0
DIA        0
1200551    0
1210622    0
1200567    0
1200575    0
1200545    0
1210702    0
1200560    0
1210683    0
1200548    0
1200570    0
1210718    0
1210734    0
1200571    0
1200579    0
1210770    0
1200558    0
1200562    0
1200554    0
dtype: int64


### DD_MED.csv
Substitui-se os valores ausentes pela mediana circular

In [55]:
# Função para calcular a mediana circular (em graus)
def circular_median(values):
    # Converte os valores para radianos
    radians = np.deg2rad(values)
    # Calcula o seno e cosseno da média
    median_radians = np.arctan2(np.mean(np.sin(radians)), np.mean(np.cos(radians)))
    # Converte de volta para graus
    return np.rad2deg(median_radians) % 360

# Caminho para o dataset
dataset_path = os.path.join('data', 'raw', 'Dados_diarios_T_Vento_Prec_capitaisDistrito_2013 a 2023.xlsx')

# Importando a folha 'DD_MED', tratando -990 como ausente
df_ddmed = pd.read_excel(dataset_path, sheet_name='DD_MED', na_values=-990)

# Verificando se existem valores ausentes antes de qualquer tratamento
print("\nResumo dos valores ausentes antes do tratamento na folha DD_MED:")
print(df_ddmed.isnull().sum())

# Se houver valores ausentes, vamos prosseguir com o tratamento:
if df_ddmed.isnull().sum().sum() > 0:
    # Calculando a mediana circular
    mediana_circular = circular_median(df_ddmed.select_dtypes(include=['float64', 'int64']).stack())

    # Preenchendo valores ausentes com a mediana circular
    df_ddmed.fillna(mediana_circular, inplace=True)

    # Resultado final após o tratamento
    print("\nResumo dos valores ausentes após o tratamento na folha DD_MED:")
    print(df_ddmed.isnull().sum())

    # Salvando o dataset tratado em Excel (opcional)
    df_ddmed.to_csv('data/processed/DD_MED_processed.csv', index=False)
else:
    print("\nNão há valores ausentes na folha DD_MED.")



Resumo dos valores ausentes antes do tratamento na folha DD_MED:
ANO           0
MES           0
DIA           0
1200551    2540
1210622    1090
1200567      36
1200575      47
1200545     617
1210702     665
1200560      85
1210683     191
1200548      63
1200570       1
1210718     207
1210734      62
1200571    1052
1200579      36
1210770     192
1200558      36
1200562     287
1200554    1026
dtype: int64

Resumo dos valores ausentes após o tratamento na folha DD_MED:
ANO        0
MES        0
DIA        0
1200551    0
1210622    0
1200567    0
1200575    0
1200545    0
1210702    0
1200560    0
1210683    0
1200548    0
1200570    0
1210718    0
1210734    0
1200571    0
1200579    0
1210770    0
1200558    0
1200562    0
1200554    0
dtype: int64


### Repartição da Produção_200100101_20250424.csv

Não tem valores ausentes

In [5]:
file_path = "data/raw/Repartição da Produção_200100101_20250424.csv"
df = pd.read_csv(file_path, encoding='utf-8', sep=';', skiprows=2)
missing_values = df.isna().sum()
print(missing_values[missing_values > 0])

Series([], dtype: int64)


## Agregação dos Dados

In [9]:
import pandas as pd
import numpy as np
from pathlib import Path

def carregar_dados_eolica(path_eolica: str) -> pd.DataFrame:
    """Carrega produção eólica, mantendo apenas Data e Hora e Eólica."""
    df = pd.read_csv(path_eolica, sep=';', skiprows=2, parse_dates=['Data e Hora'])
    df.columns = df.columns.str.strip()
    df = df[['Data e Hora', 'Eólica']]
    df.set_index('Data e Hora', inplace=True)
    df_diario = df.resample('D').sum().reset_index()
    df_diario.rename(columns={'Data e Hora': 'Data'}, inplace=True)
    return df_diario

def carregar_dd_ff(dd_path: str, ff_path: str) -> pd.DataFrame:
    dd = pd.read_csv(dd_path)
    ff = pd.read_csv(ff_path)

    # Renomear colunas para compatibilidade com pd.to_datetime
    dd = dd.rename(columns={'ANO': 'year', 'MES': 'month', 'DIA': 'day'})
    ff = ff.rename(columns={'ANO': 'year', 'MES': 'month', 'DIA': 'day'})

    # Criar coluna de data
    dd['Data'] = pd.to_datetime(dd[['year', 'month', 'day']])
    ff['Data'] = pd.to_datetime(ff[['year', 'month', 'day']])

    # Estações são colunas com nomes numéricos
    estacoes = [col for col in dd.columns if col.isdigit()]

    dd_rad = np.deg2rad(dd[estacoes])
    x = ff[estacoes] * np.cos(dd_rad)
    y = ff[estacoes] * np.sin(dd_rad)

    x_medio = x.mean(axis=1)
    y_medio = y.mean(axis=1)

    intensidade_media = np.sqrt(x_medio**2 + y_medio**2)
    direcao_media = (np.rad2deg(np.arctan2(y_medio, x_medio))) % 360

    df_nacional = pd.DataFrame({
        'Data': dd['Data'],
        'Intensidade_Media': intensidade_media,
        'Direcao_Media': direcao_media
    })
    return df_nacional

def carregar_temperatura(temp_path: str) -> pd.DataFrame:
    temp = pd.read_csv(temp_path)

    # Renomear colunas
    temp = temp.rename(columns={'ANO': 'year', 'MES': 'month', 'DIA': 'day'})
    temp['Data'] = pd.to_datetime(temp[['year', 'month', 'day']])

    estacoes = [col for col in temp.columns if col.isdigit()]
    temp_media = temp[estacoes].mean(axis=1)

    df_temp = pd.DataFrame({
        'Data': temp['Data'],
        'Temperatura_Media': temp_media
    })
    return df_temp


def main():
    path_eolica = 'data/raw/Repartição da Produção_200100101_20250424.csv'
    path_dd = 'data/processed/DD_MED_processed.csv'
    path_ff = 'data/processed/FF_MED_processed.csv'
    path_temp = 'data/processed/T_MED_processed.csv'
    pasta_saida = 'data/processed/'

    # Carregar datasets
    df_eolica = carregar_dados_eolica(path_eolica)
    df_vento = carregar_dd_ff(path_dd, path_ff)
    df_temp = carregar_temperatura(path_temp)

    # Juntar meteorologia (vento + temperatura)
    df_meteo = pd.merge(df_vento, df_temp, on='Data')

    # Juntar meteorologia + produção eólica
    df_final = pd.merge(df_meteo, df_eolica, on='Data', how='left')

    Path(pasta_saida).mkdir(parents=True, exist_ok=True)
    df_final.to_csv(Path(pasta_saida) / 'meteorologia_eolica.csv', index=False, sep=';')

    print(df_final.head())

if __name__ == "__main__":
    main()


        Data  Intensidade_Media  Direcao_Media  Temperatura_Media    Eólica
0 2013-01-01           1.629318     327.262439           9.488889   89358.2
1 2013-01-02           1.564218      31.852420           8.683333  118865.5
2 2013-01-03           2.676138      79.706195           9.277778  232257.7
3 2013-01-04           1.996991      88.710336           8.761111  149543.7
4 2013-01-05           1.045090      83.465882           8.600000   23830.8
